In [1]:
from datetime import datetime
from netCDF4 import Dataset as ds
import numpy as np
import dateutil as du 
import parser
import pytz
import dateutil.parser 
import pandas as pd
from pandas import DataFrame
import csv
import glob, os
from os.path import expanduser
from pathlib import Path
import xarray as xr 

In [2]:
df = pd.read_csv('df_adjusted_date.csv')

In [3]:
# Change days to using adjusted date
df['adjusted_date'] = df['adjusted_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df['DAY'] = df['adjusted_date'].dt.day
df['MONTH'] = df['adjusted_date'].dt.month
df['YEAR'] = df['adjusted_date'].dt.year

In [4]:
# insert '0' and convert to string
df['DAY'] = df['DAY'].apply(lambda x: int(float(x)))
df['DAY'] = df['DAY'].apply('{0:0>2}'.format)
df['MONTH'] = df['MONTH'].apply(lambda x: int(float(x)))
df['MONTH'] = df['MONTH'].apply('{0:0>2}'.format)

In [5]:
df.head(2)

,Unnamed: 0,ITEM_ID,COUNTRY,lat,lon,SEVERITY_CODE,DAY,MONTH,YEAR,full_date,rank_date,JD,MMM_date,adjusted_date
0,358,358,United States,28.1648,-94.2983,1,16,08,1993,1993-10-11,16-10-1993,284,1993-08-16,1993-08-16
1,384,384,United States,28.1648,-94.2983,1,14,05,1994,1994-05-14,16-5-1994,134,1994-07-16,1994-05-14


In [6]:
# Function to slice the filename with .nc extention, this will be used to open the files in directory
def file_info(ncfilename):
    with ds(ncfilename,'r') as ncdat:
        #string = str(ncfilename)
        #filename = string[-16:] 
        array = xr.open_dataset(ncfilename) # open_dataset works fine with either full path or filename.nc 
        return array  
def array_sel(array):
    dsloc = array.sel(lon=lon,lat=lat,time=time,method='nearest') 
    dhw = dsloc['degree_heating_week']
    return dhw 

In [7]:
home = Path(expanduser("~"))
x = []
for row in df.itertuples():
        lat = row.lat
        lon = row.lon
        time = row.adjusted_date
        path = home / Path("D:/PGonzalez/DHW/DHW_monmax/dhw_1985-2017_runmax30.nc") 
        array = file_info(path)
        dhw_val = array_sel(array)
        x.append(dhw_val)
dhw = [item.values for item in x]
df['DHW'] = dhw
df.to_csv('dhw_adjusteddaterunmax30.csv')

In [ ]:
x[21002].values

In [ ]:
dhw =[]
for item in x:
    values = item.values
    dhw.append(values)
    #print(len(dhw))
df['DHW'] = dhw

In [ ]:
df.to_csv('dhw_adjusteddate.csv')

In [ ]:
filename = 'dhw_1985-2017.nc'   
array = xr.open_dataset(filename)  
dsloc = array.sel(lon=-94.2983,lat=28.1648,time='1993-11-01',method='nearest') 
dhw = dsloc['degree_heating_week']

In [ ]:
dhw